In [3]:
import numpy as np
import pandas as pd
import time
import random
import matplotlib.pyplot as plt


In [4]:
###################################################################################################################################################################
#HMA_pth  = r"/Users/yuhan/Desktop/paper——revision/FLCounty_JEPR/HazardMitigationGrant/HazardMitigationAssistanceMitigatedProperties1.csv"
disaster_pth  = r"/Users/yuhan/Desktop/paper——revision/FLCounty_JEPR/HazardMitigationGrant/DisasterDeclarationsSummaries.csv"
DisasterData = pd.read_csv( disaster_pth )
#print( DisasterData.loc[2] )
DisasterData["incidentType"].unique()

array(['Severe Storm', 'Biological', 'Tropical Storm', 'Hurricane',
       'Winter Storm', 'Severe Ice Storm', 'Typhoon', 'Tornado', 'Fire',
       'Flood', 'Snowstorm', 'Other', 'Earthquake', 'Volcanic Eruption',
       'Toxic Substances', 'Drought', 'Dam/Levee Break', 'Freezing',
       'Coastal Storm', 'Fishing Losses', 'Mud/Landslide', 'Human Cause',
       'Terrorist', 'Tsunami', 'Chemical'], dtype=object)

In [5]:

#HazardMitigationAssistanceProjectsByNfipCrsCommunities.csv
# #MitigationProjectsFL = ProjectData.loc[ProjectData["state"] == "Florida"].reset_index()
# MitigationProjects1 = ProjectData
DisasterData = DisasterData.loc[DisasterData["incidentType"].isin( ['Severe Storm', 
       'Tropical Storm', 'Hurricane',
       'Winter Storm', 'Severe Ice Storm', 'Typhoon', 'Tornado', 'Fire',
       'Flood', 'Snowstorm', 'Other', 'Earthquake', 'Volcanic Eruption',
       'Toxic Substances', 'Drought', 'Dam/Levee Break', 'Freezing',
       'Coastal Storm', 'Fishing Losses', 'Mud/Landslide',
       'Terrorist', 'Tsunami']  )].reset_index()

DeclaredDisasters = DisasterData.loc[ :,["disasterNumber", "fyDeclared",
                                    "disasterType", "incidentType", "incidentBeginDate",
                                    "incidentEndDate", "declaredCountyArea" ]]

DeclaredDisasters = DeclaredDisasters.drop_duplicates()

DeclaredDisasters["incidentBeginDate"]  = pd.to_datetime( DeclaredDisasters["incidentBeginDate"], errors = "coerce" )
DeclaredDisasters["incidentEndDate"]    = pd.to_datetime( DeclaredDisasters["incidentEndDate"], errors = "coerce" )
DeclaredDisasters["duration"]           = np.abs( (DeclaredDisasters["incidentEndDate"] - 
                                                    DeclaredDisasters["incidentBeginDate"]).dt.total_seconds() / 60 / 60 / 24 ) + 1

DeclaredDisasters["BeginQuarter"]  = (DeclaredDisasters["incidentBeginDate"].dt.month-1)//2 + 1

DeclaredDisasters["incidentdate"]       =  DeclaredDisasters["incidentBeginDate"].dt.year.astype(str).str.zfill(4) \
                                            + DeclaredDisasters["incidentBeginDate"].dt.month.astype(str).str.zfill(2)

DeclaredDisasters['disasterNumber']     =  DeclaredDisasters['disasterNumber'].astype("int64")


DeclaredDisasters = DeclaredDisasters.loc[ DeclaredDisasters["fyDeclared"] > 1988 ]
DeclaredDisasters = DeclaredDisasters.loc[ :,["disasterNumber", "fyDeclared",
                                              "disasterType", "incidentType", 
                                              "duration", "incidentdate" ]].drop_duplicates()

DeclaredDisasters = DeclaredDisasters.drop_duplicates()


#print( DeclaredDisasters.iloc[1] )
print( DeclaredDisasters.shape )
print( "The length declared is ", len( np.unique( DeclaredDisasters.incidentdate ) ) )


#########################################################################################################################

sheldus_pth  = r"/Users/yuhan/Desktop/paper——revision/UID10427_ZIP_2/UID10427f_AGG_A_month.csv"
SheldusData = pd.read_csv( sheldus_pth )
#print( SheldusData.iloc[2] )
SheldusData = SheldusData.loc[ SheldusData["Year"] > 1988 ]
SheldusData["County FIPS"]      =  SheldusData["County FIPS"].str.strip("'")

SheldusData["Quarter"]          = (SheldusData["Month"]-1)//2 + 1

#SheldusData["incidentdate"]     =  SheldusData["Year"].astype(str).str.zfill(4) \
#                                + SheldusData["Month"].astype(str).str.zfill(2) 

SheldusData["sheldusdateplace"] =  SheldusData["Year"].astype(str).str.zfill(4) \
                                + SheldusData["Month"].astype(str).str.zfill(2) \
                                + SheldusData["County FIPS"]

SheldusData["sheldusyearplace"] =  SheldusData["Year"].astype(str).str.zfill(4) \
                                + SheldusData["County FIPS"]

print( "The length sheldusdateplace is ", len( np.unique( SheldusData.sheldusyearplace ) ) )
print( "total number of hazards is ", len( SheldusData ) )


#wrong
#SheldusDeclaredDisasters = pd.merge( SheldusData, DeclaredDisasters ,
#                   left_on=['incidentdate'], right_on=['incidentdate'], how='left')


DeclaredDisasters.to_csv(r"/Users/yuhan/Desktop/paper——revision/DeclaredDisasters.csv", index=False)



#wrong again
#SheldusDeclaredDisasters.to_csv(r"/Users/yuhan/Desktop/paper——revision/SheldusDeclaredDisasters.csv", index=False)

####################################################################################################################################################################


(3445, 6)
The length declared is  392
The length sheldusdateplace is  84255
total number of hazards is  452627


In [5]:
SheldusData.Hazard.unique()

array(['Lightning', 'Winter Weather', 'Severe Storm/Thunder Storm',
       'Wind', 'Hail', 'Flooding', 'Heat', 'Tornado',
       'Hurricane/Tropical Storm', 'Fog', 'Coastal', 'Landslide',
       'Wildfire', 'Drought', 'Avalanche', 'Tsunami/Seiche', 'Earthquake',
       'Volcano'], dtype=object)

In [6]:
''' 'Geophysical',
'Hydrological',
'Meteorological',
'Climatological' '''

def label_hazard(row):
   if 'Severe Storm/Thunder Storm' in row['Hazard' ]  :
      return 'Meteorological'
   if 'Flooding' in row['Hazard' ] :
      return 'Hydrological'
   if 'Drought' in row['Hazard' ] :
      return 'Climatological'
   if 'Wind' in row['Hazard' ]:
      return 'Meteorological'
   if 'Winter Weather' in row['Hazard' ] :
      return 'Climatological'
   if 'Coastal' in row['Hazard' ]:
      return 'Hydrological'
   if 'Hurricane/Tropical Storm' in row['Hazard' ] :
      return 'Meteorological'
   if 'Lightning' in row['Hazard' ]:
      return 'Meteorological'
   if 'Hail' in row['Hazard' ]:
      return 'Climatological'
   if 'Tornado' in row['Hazard' ] :
      return 'Meteorological'
   if 'Wildfire' in row['Hazard' ] :
      return 'Climatological'
   if 'Earthquake' in row['Hazard' ]:
      return 'Geophysical'
   if 'Volcano' in row['Hazard' ] :
      return 'Geophysical'
   if 'Heat' in row['Hazard' ]:
      return 'Climatological'
   if 'Landslide' in row['Hazard' ] :
      return 'Geophysical'
   if 'Fog' in row['Hazard' ] :
      return 'Meteorological'
   if 'Avalanche' in row['Hazard' ] :
      return 'Geophysical'
   if 'Tsunami/Seiche' in row['Hazard' ] :
      return 'Geophysical'
   return 'Other'

''' ['Lightning', 'Winter Weather', 'Severe Storm/Thunder Storm',
       'Wind', 'Hail', 'Flooding', 'Heat', 'Tornado',
       'Hurricane/Tropical Storm', 'Fog', 'Coastal', 'Landslide',
       'Wildfire', 'Drought', 'Avalanche', 'Tsunami/Seiche', 'Earthquake',
       'Volcano'] '''


SheldusData[ 'Hazard2' ] = SheldusData.apply (lambda row: label_hazard(row), axis=1)
print( SheldusData[ 'Hazard2' ].value_counts() )
print(SheldusData.columns)



Hazard2
Meteorological    314410
Climatological     80668
Hydrological       55003
Geophysical         2546
Name: count, dtype: int64
Index(['State Name', 'County Name', 'County FIPS', 'Hazard', 'Year', 'Month',
       'CropDmg', 'CropDmg(ADJ 2021)', 'CropDmgPerCapita(ADJ 2021)',
       'PropertyDmg', 'PropertyDmg(ADJ 2021)',
       'PropertyDmgPerCapita(ADJ 2021)', 'Injuries', 'InjuriesPerCapita',
       'Fatalities', 'FatalitiesPerCapita', 'Duration_Days',
       'Fatalities_Duration', 'Injuries_Duration', 'Property_Damage_Duration',
       'Crop_Damage_Duration', 'Records', 'Quarter', 'sheldusdateplace',
       'sheldusyearplace', 'Hazard2'],
      dtype='object')


In [7]:
print( SheldusData['State Name'].unique() )
print("*************")
print( SheldusData['Year'].unique() )


['ALABAMA' 'ARIZONA' 'ARKANSAS' 'CALIFORNIA' 'COLORADO' 'CONNECTICUT'
 'DELAWARE' 'DISTRICT OF COLUMBIA' 'FLORIDA' 'GEORGIA' 'HAWAII' 'IDAHO'
 'ILLINOIS' 'NEW YORK' 'NORTH CAROLINA' 'NORTH DAKOTA' 'OHIO' 'OKLAHOMA'
 'INDIANA' 'IOWA' 'KANSAS' 'KENTUCKY' 'LOUISIANA' 'MAINE' 'MARYLAND'
 'MASSACHUSETTS' 'MICHIGAN' 'MINNESOTA' 'MISSISSIPPI' 'MISSOURI' 'MONTANA'
 'NEBRASKA' 'NEVADA' 'NEW HAMPSHIRE' 'NEW JERSEY' 'NEW MEXICO' 'OREGON'
 'PENNSYLVANIA' 'RHODE ISLAND' 'SOUTH CAROLINA' 'SOUTH DAKOTA' 'TENNESSEE'
 'TEXAS' 'UTAH' 'VERMONT' 'VIRGINIA' 'WASHINGTON' 'WEST VIRGINIA'
 'WISCONSIN' 'WYOMING' 'ALASKA' 'PUERTO RICO' 'AMERICAN SAMOA'
 'VIRGIN ISLANDS' 'GUAM' 'COMMONWEALTH OF THE NORTHERN MARIANA ISLANDS']
*************
[1990 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005
 2006 2007 2008 2009 2010 2011 2012 1989 1991 1992 2013 2014 2015 2016
 2017 2018 2019 2020 2021]


In [8]:
print( SheldusData['County FIPS'].value_counts() )


County FIPS
48201    600
19153    555
28049    552
36029    540
06071    539
        ... 
02158      1
51123      1
02160      1
02260      1
02120      1
Name: count, Length: 3251, dtype: int64


In [9]:
#SheldusData  = SheldusData.loc[ SheldusData['Year'] > 1989 ]
SheldusData2 = SheldusData.groupby([ 'State Name', 'County FIPS',   
                                    'Hazard2', 'Year']).agg({
                   # 'Hazard'                            :  ', '.join ,
                    'CropDmg'                           :  ['sum'], 
                    'CropDmg(ADJ 2021)'                 :  ['sum'],
                    'CropDmgPerCapita(ADJ 2021)'        :  ['sum'],
                    'PropertyDmg'                       :  ['sum'],
                    'PropertyDmg(ADJ 2021)'             :  ['sum'],
                    'PropertyDmgPerCapita(ADJ 2021)'    :  ['sum'],
                    'Injuries'                          :  ['sum'],
                    'InjuriesPerCapita'                 :  ['sum'],
                    'Fatalities'                        :  ['sum'],
                    'FatalitiesPerCapita'               :  ['sum'],
                    'Duration_Days'                     :  ['max'],
                    'Fatalities_Duration'               :  ['max'],
                    'Injuries_Duration'                 :  ['max'],
                    'Property_Damage_Duration'          :  ['max'],
                    'Crop_Damage_Duration'              :  ['max']  }).reset_index()

SheldusData2.columns = SheldusData2.columns.droplevel(1)

#print( SheldusData2.head() )
print("**********")
print( SheldusData2.tail() )
print("**********")

print( SheldusData2.shape )

**********
       State Name County FIPS         Hazard2  Year  CropDmg  \
148548    WYOMING       56045  Meteorological  2009      0.0   
148549    WYOMING       56045  Meteorological  2010      0.0   
148550    WYOMING       56045  Meteorological  2011      0.0   
148551    WYOMING       56045  Meteorological  2013      0.0   
148552    WYOMING       56045  Meteorological  2015      0.0   

        CropDmg(ADJ 2021)  CropDmgPerCapita(ADJ 2021)  PropertyDmg  \
148548                0.0                         0.0      19000.0   
148549                0.0                         0.0     190000.0   
148550                0.0                         0.0     260500.0   
148551                0.0                         0.0      10000.0   
148552                0.0                         0.0      46000.0   

        PropertyDmg(ADJ 2021)  PropertyDmgPerCapita(ADJ 2021)  Injuries  \
148548               23312.80                         3.32612       0.0   
148549              229365.76    

In [10]:
SheldusData2.to_csv(r"/Users/yuhan/Desktop/paper——revision/SheldusData_add.csv", index=False)


In [44]:
'''
def label_hazard0(row):
   if row['Hazard' ] == 'Severe Storm/Thunder Storm' :
      return 'Severe Storm/Hurricane/Tornado'
   if row['Hazard' ] == 'Flooding':
      return 'Flooding'
   if row['Hazard' ] == 'Drought' :
      return 'Drought/Heat'
   if row['Hazard' ] == 'Wind':
      return 'Wind'
   if row['Hazard' ]  == 'Winter Weather':
      return 'Winter Weather'
   if row['Hazard' ] == 'Coastal':
      return 'Severe Storm/Hurricane/Tornado'
   if row['Hazard' ] == 'Hurricane/Tropical Storm':
      return 'Severe Storm/Hurricane/Tornado'
   if row['Hazard' ] == 'Lightning':
      return 'Severe/Thunder Storm'
   if row['Hazard' ] == 'Hail':
      return 'Winter Weather'
   if row['Hazard' ] == 'Tornado':
      return 'Severe Storm/Hurricane/Tornado'
   if row['Hazard' ] == 'Wildfire':
      return 'Wildfire'
   if row['Hazard' ] == 'Earthquake':
      return 'Earthquake/Volcano/Landslide'
   if row['Hazard' ] == 'Volcano':
      return 'Earthquake/Volcano/Landslide'
   if row['Hazard' ] == 'Heat':
      return 'Drought/Heat'
   if row['Hazard' ] == 'Landslide':
      return 'Earthquake/Volcano/Landslide'
   """ if row['Hazard' ] == 'Tsunami/Seiche':
      return 'Hurricane' """
   return 'Other Hazard'

#SheldusData[ 'Hazard0' ] = SheldusData.apply (lambda row: label_hazard(row), axis=1)
#print( SheldusData[ 'Hazard2' ].value_counts() )
'''

In [39]:
HMA_pth2 =  r"/Users/yuhan/Desktop/paper——revision/FLCounty_JEPR/HazardMitigationGrant/HazardMitigationAssistanceProjects_new.csv"
HMAData2 = pd.read_csv( HMA_pth2 )

HMAData2["dateApproved"]    = pd.to_datetime( HMAData2["dateApproved"], errors = "coerce" )
HMAData2["ApprovedYear"]    = HMAData2["dateApproved"].dt.year

HMAData2.head()

,projectIdentifier,programArea,programFy,region,state,stateNumberCode,county,countyCode,disasterNumber,projectCounties,...,subrecipientAdminCostAmt,srmcObligatedAmt,recipientAdminCostAmt,costSharePercentage,benefitCostRatio,netValueBenefits,numberOfFinalProperties,numberOfProperties,id,ApprovedYear
0,DR-4339-0169-R,HMGP,2017,2,Puerto Rico,72,Bayamon,21.0,4339.0,BAYAMON,...,0.0,67315.31,0.0,1.00,2.952,4771891.0,1,5,b6d7dd10-e1be-4254-90c9-cbcc0a4207f1,2022.0
1,DR-1606-0008-F,HMGP,2005,6,Texas,48,Jefferson,245.0,1606.0,JEFFERSON,...,450.0,0.00,272.0,0.75,0.000,0.0,0,0,fbdc48fd-3910-4510-ac6c-a26243538699,2006.0
2,DR-4283-0041-R,HMGP,2017,4,Florida,12,Broward,11.0,4283.0,BROWARD,...,0.0,0.00,0.0,0.75,1.305,61193.0,1,1,89910914-33e7-48eb-b8a4-d6446d906684,2021.0
3,DR-1913-0005-M,HMGP,2010,1,New Hampshire,33,Statewide,0.0,1913.0,STATEWIDE,...,0.0,0.00,0.0,1.00,0.000,0.0,0,0,3b737383-99b8-451d-a1c9-6c8de5be2cbe,2016.0
4,DR-1799-0002-M,HMGP,2009,1,New Hampshire,33,Statewide,0.0,1799.0,STATEWIDE,...,0.0,0.00,0.0,1.00,0.000,0.0,0,0,3f15c474-f0b0-45a2-9046-dbe6b21ffded,2010.0


In [40]:
print( HMAData2[ ["ApprovedYear", "programFy"] ] )

print( HMAData2.programFy.unique() )
print( HMAData2.ApprovedYear.unique() )

       ApprovedYear  programFy
0            2022.0       2017
1            2006.0       2005
2            2021.0       2017
3            2016.0       2010
4            2010.0       2009
...             ...        ...
32555        2005.0       2005
32556        2015.0       2014
32557        2007.0       2007
32558        2013.0       2012
32559        2016.0       2015

[32560 rows x 2 columns]
[2017 2005 2010 2009 2013 2015 2018 2021 2004 2011 2014 2001 2020 2000
 2002 2016 2003 2012 2019 2006 1999 2022 2008 2007 1997 1998 1994 1995
 1991 1990 1992 1989 1996 1993 2023]
[2022. 2006. 2021. 2016. 2010. 2019. 2020. 2018. 2014. 2012. 2011. 2015.
 2013. 2004. 2017. 2023. 2007. 2005. 2008. 2003. 2009. 1999. 2000. 2001.
 1998. 1997. 2002. 1993. 1991. 1996. 1994. 1992. 1995.   nan 1990.]


In [41]:
HMAData2['dateClosed'].fillna( pd.to_datetime("today"), inplace=True )
HMAData2["dateClosed"]      = pd.to_datetime( HMAData2["dateClosed"], errors = "coerce" )
HMAData2["duration_years"]  = np.abs( (HMAData2["dateClosed"] - 
                                        HMAData2["dateApproved"]).dt.total_seconds() / 60 / 60 / 24/ 365 )
HMAData2['stateCode']       =  HMAData2['stateNumberCode'].astype(str).str.zfill(2)
HMAData2['countyCode']      =  HMAData2['countyCode'].fillna(0).astype(int).astype(str).str.zfill(3)
HMAData2['FIPS']            =  HMAData2['stateCode']  + HMAData2['countyCode']

print( HMAData2.loc[ ( HMAData2['countyCode'] == '000' )&
              ( HMAData2['recipient']  == 'Statewide' ) ].shape)

print( HMAData2.shape )

(1972, 35)
(32560, 35)


In [42]:
HMAData2['FIPS'] .value_counts()

FIPS
06000    489
06037    376
12000    352
48201    233
72000    201
        ... 
06031      1
48325      1
51137      1
47111      1
20193      1
Name: count, Length: 2928, dtype: int64

In [43]:
#HMAData3 = HMAData2.dropna( subset=['duration_years', 'countyCode', 'disasterNumber'] )
HMAData3 = HMAData2

HMAData3['stateNumberCode']  = HMAData3['stateNumberCode'].apply(int).apply(str)
HMAData3['countyCode']       = HMAData3['countyCode'].apply(int).apply(str)
HMAData3['FIPS2']            = HMAData3['stateNumberCode'].str.zfill(2) + HMAData3['countyCode'].str.zfill(3)

print( HMAData3.shape )
print( HMAData3.FIPS.isin( HMAData3.FIPS2 ).astype(int).sum() )


(32560, 36)
32560


In [44]:
HMAData3['FIPS'].compare( HMAData3[ 'FIPS2' ].copy() )


,self,other


In [45]:
HMAData3['disasterNumber'].min()
print( HMAData3.shape )
print( DeclaredDisasters.shape )


(32560, 36)
(3445, 6)


In [46]:
HMAData3['disasterNumber']  = HMAData3['disasterNumber'].fillna(0).astype("int64")
#pd.to_numeric( HMAData3['disasterNumber'] , errors='coerce').astype("int64")

new_df3 = pd.merge(HMAData3, DeclaredDisasters, left_on=['disasterNumber'], 
                   right_on=['disasterNumber'], how='outer')

print( new_df3.shape )

new_df3["incidentdateplace"] = new_df3["incidentdate"] + new_df3['FIPS']
#########################################################################################################################
#########################################################################################################################
new_df4 = new_df3.loc[ new_df3["FIPS"] != "0" ]
print( new_df4.shape )


(34264, 41)
(34264, 42)


In [47]:
new_df3["FIPS"].unique()

array(['72021', '72003', '72015', ..., '02240', '38085', nan],
      dtype=object)

In [81]:
new_df4.programFy.unique()

array([2017., 2005., 2010., 2009., 2013., 2015., 2018., 2021., 2004.,
       2011., 2014., 2001., 2020., 2000., 2002., 2016., 2003., 2012.,
       2019., 2006., 2008., 2007., 2022., 1998., 1999., 1997., 1996.,
       1994., 1995., 1991., 1990., 1992., 1989., 1993., 2023.,   nan])

In [80]:

new_df4.fyDeclared.unique()

array([2017., 2005., 2010., 2009., 2013., 2015., 2018., 2021., 2004.,
       2011., 2014., 2001., 2020., 2000., 2002., 2016., 2003., 2012.,
       2019., 2006.,   nan, 1999., 2022., 2008., 2007., 1997., 1998.,
       1994., 1995., 1991., 1990., 1992., 1989., 1996., 1993., 2023.])

In [67]:
new_df4['Year'] = new_df4.programFy.fillna(new_df4.fyDeclared)
print( new_df4['Year'].unique() )

[2017. 2005. 2010. 2009. 2013. 2015. 2018. 2021. 2004. 2011. 2014. 2001.
 2020. 2000. 2002. 2016. 2003. 2012. 2019. 2006. 2008. 2007. 2022. 1998.
 1999. 1997. 1996. 1994. 1995. 1991. 1990. 1992. 1989. 1993. 2023.]


In [68]:
new_df5 = new_df4.groupby([ 'FIPS', 'Year', 'programArea', 'disasterNumber' ]).agg({
                    'projectAmount'                           :  ['sum'], 
                    'numberOfFinalProperties'                 :  ['sum'],
                    'dateInitiallyApproved'        :  ['count'],
  }).reset_index()

new_df5.columns = new_df5.columns.droplevel(1)
new_df5.columns = [ "FIPS", "Year", "programArea", "disasternumber", 
                   "HMAAmount", "HMANprop", "HMAcount" ]


In [69]:
new_df5.head()

,FIPS,Year,programArea,disasternumber,HMAAmount,HMANprop,HMAcount
0,01000,1990.0,HMGP,848,31588.0,0.0,0
1,01000,1990.0,HMGP,856,138888.0,0.0,0
2,01000,1990.0,HMGP,861,169166.0,0.0,0
3,01000,1994.0,HMGP,1013,356892.0,0.0,0
4,01000,1994.0,HMGP,1019,267651.0,0.0,0


In [70]:
print( new_df5['FIPS'].isnull().sum() )
print( SheldusData2['County FIPS'].isnull().sum() )
print(SheldusData2.columns)

0
0
Index(['State Name', 'County FIPS', 'Hazard2', 'Year', 'CropDmg',
       'CropDmg(ADJ 2021)', 'CropDmgPerCapita(ADJ 2021)', 'PropertyDmg',
       'PropertyDmg(ADJ 2021)', 'PropertyDmgPerCapita(ADJ 2021)', 'Injuries',
       'InjuriesPerCapita', 'Fatalities', 'FatalitiesPerCapita',
       'Duration_Days', 'Fatalities_Duration', 'Injuries_Duration',
       'Property_Damage_Duration', 'Crop_Damage_Duration'],
      dtype='object')


In [71]:
len( SheldusData2['County FIPS'].unique() ) - len( new_df5['FIPS'].unique() )

323

In [72]:
print( SheldusData2.columns )
print( new_df5.columns )

Index(['State Name', 'County FIPS', 'Hazard2', 'Year', 'CropDmg',
       'CropDmg(ADJ 2021)', 'CropDmgPerCapita(ADJ 2021)', 'PropertyDmg',
       'PropertyDmg(ADJ 2021)', 'PropertyDmgPerCapita(ADJ 2021)', 'Injuries',
       'InjuriesPerCapita', 'Fatalities', 'FatalitiesPerCapita',
       'Duration_Days', 'Fatalities_Duration', 'Injuries_Duration',
       'Property_Damage_Duration', 'Crop_Damage_Duration'],
      dtype='object')
Index(['FIPS', 'Year', 'programArea', 'disasternumber', 'HMAAmount',
       'HMANprop', 'HMAcount'],
      dtype='object')


In [73]:
#SheldusData_agg.to_csv(r"/Users/yuhan/Desktop/paper——revision/SheldusData_agg.csv", index=False)
#SheldusData_agg.to_csv(r"/Users/yuhan/Desktop/paper——revision/SheldusData_agg.csv", index=False)

AggregateData4 = pd.merge( SheldusData2, new_df5,
                   right_on = ['FIPS', 'Year'], left_on = ['County FIPS', 'Year'], how='left')

print( AggregateData4.columns )

AggregateData4 = AggregateData4.fillna(0)

Index(['State Name', 'County FIPS', 'Hazard2', 'Year', 'CropDmg',
       'CropDmg(ADJ 2021)', 'CropDmgPerCapita(ADJ 2021)', 'PropertyDmg',
       'PropertyDmg(ADJ 2021)', 'PropertyDmgPerCapita(ADJ 2021)', 'Injuries',
       'InjuriesPerCapita', 'Fatalities', 'FatalitiesPerCapita',
       'Duration_Days', 'Fatalities_Duration', 'Injuries_Duration',
       'Property_Damage_Duration', 'Crop_Damage_Duration', 'FIPS',
       'programArea', 'disasternumber', 'HMAAmount', 'HMANprop', 'HMAcount'],
      dtype='object')


In [76]:
MitigationProjects3 = AggregateData4.loc[ (AggregateData4["Year"] >= 1989) & 
                                   (AggregateData4["programArea"] != "PDM")].reset_index()

MitigationProjects3.to_csv(r"/Users/yuhan/Desktop/paper——revision/HMA_nPMD.csv", index=False)



#########################################################################################################################
MitigationProjects2 = new_df4.loc[ (new_df4["programFy"] >= 1989.0) & 
                                   (new_df4["programArea"] == "PDM")].reset_index()

MitigationProjects2.to_csv(r"/Users/yuhan/Desktop/paper——revision/HMA_PMD.csv", index=False)

In [77]:
new_df4.columns
print( new_df4.programArea.unique() )
print( MitigationProjects2.shape )


['HMGP' 'PDM' 'FMA' 'LPDM' 'BRIC' 'RFC' 'SRL' nan]
(3944, 44)


In [79]:
MitigationProjects2

,index,projectIdentifier,programArea,programFy,region,state,stateNumberCode,county,countyCode,disasterNumber,...,stateCode,FIPS,FIPS2,fyDeclared,disasterType,incidentType,duration,incidentdate,incidentdateplace,Year
0,1844,PDMC-PL-07-NE-2014-002,PDM,2014.0,7.0,Nebraska,31,Knox,107,0,...,31,31107,31107,NaN,NaN,NaN,NaN,NaN,NaN,2014.0
1,1845,PDMC-PJ-06-OK-2014-001,PDM,2014.0,6.0,Oklahoma,40,NaN,0,0,...,40,40000,40000,NaN,NaN,NaN,NaN,NaN,NaN,2014.0
2,1846,PDMC-PL-05-WI-2016-002,PDM,2016.0,5.0,Wisconsin,55,Florence,37,0,...,55,55037,55037,NaN,NaN,NaN,NaN,NaN,NaN,2016.0
3,1847,PDMC-PL-03-VA-2016-001,PDM,2016.0,3.0,Virginia,51,Waynesboro (city),820,0,...,51,51820,51820,NaN,NaN,NaN,NaN,NaN,NaN,2016.0
4,1848,PDMC-PJ-04-FL-2003-021,PDM,2003.0,4.0,Florida,12,Broward,11,0,...,12,12011,12011,NaN,NaN,NaN,NaN,NaN,NaN,2003.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3939,32550,DR-9075-0001-R,PDM,2003.0,1.0,Connecticut,9,NaN,0,9075,...,09,09000,09000,NaN,NaN,NaN,NaN,NaN,NaN,2003.0
3940,32551,DR-9428-F001-F,PDM,2004.0,8.0,North Dakota,38,Sioux,85,9428,...,38,38085,38085,NaN,NaN,NaN,NaN,NaN,NaN,2004.0
3941,32552,DR-9426-F001-F,PDM,2004.0,10.0,Oregon,41,Lane,39,9426,...,41,41039,41039,NaN,NaN,NaN,NaN,NaN,NaN,2004.0
3942,32555,DR-9018-0001-R,PDM,2002.0,4.0,North Carolina,37,NaN,0,9018,...,37,37000,37000,NaN,NaN,NaN,NaN,NaN,NaN,2002.0
